In [ ]:
# Cell 1: Install dependencies
%pip install transformers torch sentence-transformers faiss-cpu flask pyngrok flask-ngrok

In [ ]:
# Cell 2: Import libraries and setup
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify # Added Flask imports here
from flask_ngrok import run_with_ngrok # Added flask-ngrok import here
from pyngrok import ngrok # Added pyngrok import here


In [ ]:
# Cell 3: Knowledge base data (your documents as Python data)
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": """To create a Shoplite account, users visit the registration page and provide their email address, password, and basic profile information. Email verification is required within 24 hours of registration. Users can choose between buyer accounts (free) or seller accounts (requires business verification). Buyer accounts allow immediate shopping access, while seller accounts undergo a 2-3 business day verification process. All users must agree to Shoplite's terms of service and privacy policy. Password requirements include minimum 8 characters with at least one uppercase letter, one number, and one special character. Users can reset passwords via email recovery and manage notification preferences in account settings. Profile information includes name, shipping address, phone number, and communication preferences. Users can link social media accounts for faster login and can enable two-factor authentication for enhanced security. Account deletion requests are processed within 48 hours and permanently remove all personal data from Shoplite's systems."""
    },
    {
        "id": "doc2",
        "title": "Shoplite Product Search and Filtering Features",
        "content": """Shoplite's search functionality uses advanced algorithms to deliver relevant product results based on user queries. The search bar supports natural language processing, allowing users to search by product name, category, brand, or description. Advanced filtering options include price range, customer ratings, seller reputation, shipping options, and availability. Users can sort results by relevance, price (low to high or high to low), newest arrivals, popularity, and customer ratings. The search system incorporates machine learning to improve results based on user behavior and purchase history. Saved searches allow users to receive notifications when new products matching their criteria become available. Category browsing provides hierarchical navigation through product departments, subcategories, and specific product types. Image search functionality enables users to upload photos and find visually similar products."""
    },
    {
        "id": "doc3",
        "title": "Shoplite Shopping Cart and Checkout Process",
        "content": """The Shoplite shopping cart supports items from multiple sellers in a single cart with consolidated checkout. Cart features include quantity modification, item removal, save for later options, and promotional code application. The system displays real-time inventory updates and price changes before checkout. During checkout, users review cart contents, select shipping addresses, choose delivery methods, and apply payment. The process includes four steps: cart review, shipping information, payment method, and order confirmation. Users can select gift wrapping, add special delivery instructions, and choose delivery time slots where available. The cart automatically calculates taxes, shipping costs, and applicable discounts. Abandoned cart reminders are sent via email after 24 hours, and cart contents are saved for 30 days for logged-in users."""
    },
    {
        "id": "doc4",
        "title": "Shoplite Payment Methods and Security",
        "content": """Shoplite accepts multiple payment options including credit cards (Visa, MasterCard, American Express), debit cards, PayPal, Shoplite Pay, and bank transfers in selected regions. All payment processing complies with PCI DSS Level 1 security standards with 256-bit SSL encryption. Shoplite Pay allows users to store payment methods securely for faster checkout with optional biometric authentication. The platform supports installment payments through third-party providers for eligible purchases over $200. Payment security features include tokenization, 3D Secure authentication, and real-time fraud detection algorithms. Refund processing times vary by payment method: 3-5 business days for credit cards, 1-2 days for Shoplite Pay, and 5-7 days for bank transfers. Users can manage saved payment methods in their account settings with the ability to set default preferences."""
    },
    {
        "id": "doc5",
        "title": "Shoplite Order Tracking and Delivery",
        "content": """After order confirmation, users receive tracking information via email and can monitor order status through their Shoplite account. The tracking system provides real-time updates including order processing, shipping label creation, carrier pickup, transit milestones, and delivery confirmation. Delivery options include standard shipping (3-5 business days), expedited shipping (1-2 business days), and same-day delivery in eligible urban areas. International shipping is available to 50+ countries with customs documentation handled automatically. Users can modify delivery instructions before shipment and can authorize package release without signature for low-value items. Delivery issues such as failed attempts or damaged packages are handled through Shoplite's customer support with automatic rerouting or replacement options. Order history is maintained indefinitely for account holders with downloadable invoices and shipping documents."""
    },
    {
        "id": "doc6",
        "title": "Shoplite Return and Refund Policies",
        "content": """Shoplite offers a 30-day return window for most items from the delivery date, with exceptions for final sale items, personalized products, and perishable goods. Return eligibility requires items to be in original condition with tags attached and all accessories included. The return process begins through the user's order history by selecting items for return and choosing return reason. Return shipping labels are generated automatically for prepaid returns, while some cases may require customer-paid return shipping. Refunds are processed within 3-5 business days after returned items are received and inspected. Exchange options are available for size/color variations when inventory permits. Defective or incorrect items receive free return shipping and priority processing. Seasonal extensions may apply during holiday periods, and seller-specific return policies are clearly indicated on product pages."""
    },
    {
        "id": "doc7",
        "title": "Shoplite Product Reviews and Ratings System",
        "content": """The review system allows verified purchasers to rate products on a 1-5 star scale and write detailed reviews with photos and videos. Reviews must be submitted within 90 days of purchase and are moderated for inappropriate content. Helpful voting enables community members to identify useful reviews, with the most helpful reviews featured at the top. Review guidelines prohibit promotional content, offensive language, personal information, and irrelevant comments. Sellers can respond to reviews professionally to address customer concerns or provide additional information. The rating system calculates average scores based on all verified purchases and displays breakdowns by star rating. Reviewers receive loyalty points for helpful reviews that can be redeemed for future purchases. Anonymous reviewing is available for sensitive products, and users can report suspicious reviews for investigation."""
    },
    {
        "id": "doc8",
        "title": "Shoplite Seller Account Setup and Management",
        "content": """Seller registration requires business information including legal business name, tax identification number, business address, and contact details. The verification process includes business legitimacy checks, tax compliance validation, and may require additional documentation based on business type. Once approved, sellers complete store setup by configuring store policies, shipping methods, return preferences, and payment schedules. Seller dashboard provides analytics on sales performance, inventory levels, customer metrics, and financial reports. Seller tiers include Basic (free with 8% commission), Professional ($29.99/month with 5% commission), and Enterprise (custom pricing with 3% commission). Seller performance metrics evaluate order defect rate, cancellation rate, late shipment rate, and customer service response time. Performance below standards may result in account restrictions or suspension."""
    },
    {
        "id": "doc9",
        "title": "Shoplite Inventory Management for Sellers",
        "content": """Sellers can manage inventory through the web dashboard or bulk CSV uploads with support for real-time quantity updates. Inventory features include low stock alerts, automatic out-of-stock management, and variant tracking for products with multiple options. The system supports barcode scanning for physical inventory counts and integrates with popular inventory management software. Bulk editing allows simultaneous updates to prices, quantities, and product status across multiple listings. Seasonal inventory planning tools help sellers forecast demand and optimize stock levels. Fulfillment by Shoplite (FBS) options provide warehousing, packing, and shipping services for eligible sellers. Inventory performance metrics track sell-through rates, days of supply, and inventory turnover to help sellers optimize their product assortment. Backorder management allows pre-orders for out-of-stock items with clear delivery estimates."""
    },
    {
        "id": "doc10",
        "title": "Shoplite Commission and Fee Structure",
        "content": """Shoplite's commission model varies by product category: Electronics (5%), Clothing (8%), Home & Garden (7%), Books (10%), and Luxury Goods (12%). Additional fees include payment processing fees (2.9% + $0.30 per transaction), optional promoted listing fees, and subscription fees for premium seller tiers. Payouts to sellers are processed weekly with a 3-day holding period for order verification. Minimum payout threshold is $25, and sellers can choose direct deposit, PayPal, or wire transfer for international sellers. Fee disputes must be submitted within 30 days of charge, and detailed fee breakdowns are available for each transaction. Seasonal fee adjustments may apply during peak shopping periods, with advance notice provided to sellers. Volume discounts are available for sellers exceeding $10,000 in monthly sales, with commission rates reduced by 1-2%."""
    },
    {
        "id": "doc11",
        "title": "Shoplite Customer Support Procedures",
        "content": """Customer support is available 24/7 through multiple channels: live chat, email support, phone support, and social media messaging. Support tiers include General Support for common issues, Technical Support for platform problems, and Escalation Support for complex cases. Average response times are: live chat (under 2 minutes), email (4 hours), phone (immediate), with 90% of issues resolved within 24 hours. The support knowledge base contains 500+ articles and video tutorials covering common questions and procedures. Support agents follow standardized troubleshooting protocols and have access to account history and previous interactions. Customer satisfaction surveys are sent after each support interaction, with results impacting agent performance metrics. Premium support is available for enterprise customers with dedicated account managers and faster response times."""
    },
    {
        "id": "doc12",
        "title": "Shoplite Mobile App Features",
        "content": """The Shoplite mobile app (iOS and Android) offers all website functionality with additional mobile-exclusive features. App-specific features include barcode scanning for price comparison, augmented reality product visualization, and location-based store pickup options. Push notifications alert users to price drops, order updates, and personalized recommendations. Mobile wallet integration supports Apple Pay, Google Pay, and Samsung Pay for faster checkout. The app uses 30% less data than the mobile website through optimized image loading and caching. Offline mode allows browsing recently viewed products and accessing order history without internet connection. Biometric login (face ID, fingerprint) provides secure access, and the app supports dark mode for reduced eye strain. Exclusive mobile-only deals and flash sales are available to app users with geofenced promotions in specific locations."""
    },
    {
        "id": "doc13",
        "title": "Shoplite API Documentation for Developers",
        "content": """The Shoplite REST API provides programmatic access to product data, order management, and account functions. Authentication uses OAuth 2.0 with client credentials flow, requiring API keys from the developer dashboard. Rate limits are 1000 requests per hour per API key, with higher limits available for enterprise partners. Core endpoints include /products for catalog access, /orders for order management, /users for account operations, and /analytics for performance data. Webhooks support real-time notifications for order creation, shipment updates, and inventory changes. The API returns JSON responses with standardized error codes and pagination for large datasets. SDKs are available for Python, JavaScript, Java, and PHP with comprehensive documentation and code examples. Sandbox environment provides test data for development without affecting live operations."""
    },
    {
        "id": "doc14",
        "title": "Shoplite Security and Privacy Policies",
        "content": """Shoplite employs bank-level security measures including 256-bit encryption, regular security audits, and vulnerability testing. Data protection complies with GDPR, CCPA, and other regional privacy regulations with data processing agreements available. Privacy controls allow users to manage data sharing preferences, download personal data, and request data deletion. Security features include two-factor authentication, login activity monitoring, and automated fraud detection. The platform undergoes quarterly penetration testing and maintains SOC 2 Type II compliance. Data retention policies specify timeframes for storing different types of user data, with automatic deletion after inactivity periods. Security incident response procedures include immediate investigation, user notification where required, and remediation measures. Third-party data sharing is limited to essential service providers with strict contractual privacy obligations."""
    },
    {
        "id": "doc15",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": """Shoplite offers various promotional incentives including percentage discounts, fixed-amount coupons, free shipping, and buy-one-get-one deals. Promo codes can be sitewide, category-specific, brand-specific, or user-segment targeted. Automatic discounts apply at checkout without codes for seasonal sales and clearance events. Loyalty program members earn points redeemable for discounts, with tiered benefits based on annual spending. Student discounts (15% off) require .edu email verification, and military discounts (10% off) require ID.me verification. Seasonal promotions include Black Friday deals, Cyber Monday offers, back-to-school sales, and holiday specials. Referral programs provide $10 credits for both referrer and new customer after first purchase. Bundle discounts automatically apply when purchasing complementary products from the same seller."""
    },
    {
        "id": "doc16",
        "title": "Shoplite Shipping Options and Carriers",
        "content": """Shoplite partners with multiple shipping carriers including UPS, FedEx, USPS, DHL, and regional providers for international deliveries. Shipping options include economy (5-7 business days), standard (3-5 business days), expedited (1-2 business days), and overnight delivery. Free shipping is available on orders over $35 from participating sellers, with minimum spend varying by seller. International shipping costs include customs duties and taxes calculated at checkout for transparent pricing. Package consolidation combines multiple items from the same warehouse into single shipments to reduce environmental impact. Shipping protection is available for high-value items, covering loss or damage during transit. Delivery date estimates account for processing time, transit time, and carrier reliability based on historical data."""
    },
    {
        "id": "doc17",
        "title": "Shoplite Wishlist and Save for Later Features",
        "content": """Users can create multiple wishlists for different occasions (birthdays, holidays, personal shopping) with privacy controls for each list. Wishlist sharing enables sending lists to friends and family via email, social media, or direct link. Price drop notifications alert users when wished-for items decrease in price, with customizable threshold settings. "Save for Later" functionality moves cart items to a separate section for future consideration, preserving the original add-to-cart date. Wishlist analytics help users track price trends and availability of desired items. Collaborative wishlists allow multiple users to contribute items for group gifts or family shopping. Wishlist items automatically show availability status and can be moved to cart with one click during checkout."""
    },
    {
        "id": "doc18",
        "title": "Shoplite Subscription and Auto-Replenishment",
        "content": """The subscription service allows automatic recurring deliveries of frequently purchased items with flexible scheduling options. Subscription benefits include 15% discount on subscribed items, free shipping regardless of order value, and priority customer support. Users can choose delivery frequency from weekly, bi-weekly, monthly, or custom intervals, with easy modification or cancellation. The subscription management dashboard shows upcoming deliveries, past orders, and savings accumulated. Pause functionality allows temporary suspension of subscriptions during vacations or periods of non-use. Smart replenishment suggests optimal delivery timing based on usage patterns and can adjust quantities automatically. Subscription items are guaranteed in stock with backup product alternatives offered if primary items become unavailable."""
    },
    {
        "id": "doc19",
        "title": "Shoplite Gift Cards and E-Gifts",
        "content": """Shoplite gift cards are available in denominations from $10 to $500 with no expiration dates or maintenance fees. Physical gift cards ship within 2 business days with free standard shipping, while e-gifts deliver immediately via email. Gift cards can be redeemed for any products on Shoplite and can be combined with other promotions except cashback offers. Balance checking is available online or through customer service, with transaction history for each card. Corporate gift card programs offer bulk discounts and custom branding options for business clients. Gift card exchanges or refunds are available for unused cards within 30 days of purchase. Lost or stolen cards can be replaced with proof of purchase, protecting the remaining balance."""
    },
    {
        "id": "doc20",
        "title": "Shoplite Quality Assurance and Product Standards",
        "content": """All sellers must comply with Shoplite's product quality standards, which prohibit counterfeit items, misrepresented products, and prohibited materials. Product listing requirements include accurate descriptions, multiple high-quality photos from different angles, and clear specification details. Quality verification includes automated checks for listing completeness and manual reviews for high-risk categories. Customer-reported quality issues trigger investigation processes that may include product testing, seller communication, and temporary listing suspension. Product authenticity guarantees cover branded items, with third-party verification for luxury goods and electronics. Shoplite's A-to-Z Guarantee protects customers against significantly-not-as-described items, offering full refunds including shipping costs. Seller quality metrics impact search ranking and eligibility for promotional programs, encouraging maintained high standards."""
    }
]




In [ ]:
!python src/chat-interface.py https://your-tunnel.ngrok.io

In [ ]:
!mkdir -p src
!touch src/chat-interface.py

In [ ]:
# Cell 4: Load your YAML prompts as Python dictionaries
# Note: Prompts are defined as a template string in this implementation, not a separate YAML.
# Keeping the structure for consistency with the requested format.
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible",
            "Keep your answer grounded in the facts of the documents"
        ],
        "response_format": (
            "Answer: [Your response based on context]\n"
            "Sources: [List document titles referenced]"
        )
    },

    "multi_document_prompt": {
        "role": "You are a Shoplite expert assistant that synthesizes information from multiple knowledge sources.",
        "goal": "Combine and reconcile information from multiple relevant documents to provide comprehensive answers to complex questions.",
        "context_guidelines": [
            "Identify and use all relevant document snippets",
            "Synthesize information without contradicting different documents",
            "If documents contain conflicting information, prioritize the most specific or recent information",
            "Clearly indicate when information comes from multiple sources"
        ],
        "response_format": (
            "Comprehensive Answer: [Synthesized response combining all relevant documents]\n"
            "Sources: [List all document titles used with brief explanation of what information came from each]"
        )
    },

    "refusal_prompt": {
        "role": "You are a cautious Shoplite assistant that maintains strict information accuracy.",
        "goal": "Politely refuse to answer when context is insufficient or questions are outside the Shoplite knowledge base.",
        "context_guidelines": [
            "If no relevant context is found, clearly state this limitation",
            "Do not hallucinate or make up information",
            "Suggest related Shoplite topics that might be in the knowledge base",
            "Maintain helpful and professional tone"
        ],
        "response_format": (
            "Response: [Polite refusal with explanation that the information is not available in Shoplite documentation]\n"
            "Alternative Suggestions: [Related Shoplite topics user could ask about]"
        )
    },

    "clarification_prompt": {
        "role": "You are a detail-oriented Shoplite assistant that seeks clarity.",
        "goal": "Ask specific clarifying questions when user queries are ambiguous or need more specificity.",
        "context_guidelines": [
            "Identify ambiguous terms or missing information in the query",
            "Ask focused, specific clarifying questions",
            "Provide examples of what information would help answer completely",
            "Keep clarification requests minimal and relevant to Shoplite services"
        ],
        "response_format": (
            "Clarification Needed: [Specific aspect needing clarification]\n"
            "Question: [Direct clarifying question]\n"
            "Examples: [Examples of specific information that would help]"
        )
    },

    "billing_support_prompt": {
        "role": "You are a specialized Shoplite billing and payment support assistant.",
        "goal": "Provide clear, precise information about Shoplite's billing cycles, payment methods, and fee structures.",
        "context_guidelines": [
            "Quote specific amounts, timeframes, and policies from the documents",
            "Provide step-by-step instructions for payment-related procedures",
            "Be particularly careful with numerical information about fees and cycles",
            "Direct users to appropriate channels for sensitive payment information"
        ],
        "response_format": (
            "Billing Information: [Clear, precise billing/payment information]\n"
            "Specific Details: [Exact amounts, timeframes, policy references]\n"
            "Next Steps: [Actionable guidance for the user]"
        )
    }
}


In [ ]:
from flask import Flask, request, jsonify # Added Flask imports here
from flask_ngrok import run_with_ngrok # Added flask-ngrok import here
from pyngrok import ngrok # Added pyngrok import here

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    """Simple text completion - no RAG, no retrieval"""
    try:
        # Get request data
        data = request.get_json()

        if not data or 'prompt' not in data:
            return jsonify({"error": "Missing 'prompt' in request"}), 400

        prompt = data['prompt']
        max_tokens = data.get('max_tokens', 500)

        print(f"🔮 /generate endpoint called")
        print(f"   Prompt: {prompt[:100]}...")
        print(f"   Max tokens: {max_tokens}")

        # Use the same model you already loaded for RAG
        # Adjust this based on your actual model generation method
        if 'model' in globals():
            # If you have a model object already loaded
            response = model.generate(prompt, max_length=max_tokens)
        elif 'llm' in globals():
            # If you're using LangChain or similar
            response = llm(prompt)
        else:
            # Fallback - use whatever generation method you have
            # This will depend on your specific Week 3 implementation
            response = your_generation_function(prompt, max_tokens)

        return jsonify({
            "text": response,
            "model": "your-week3-model",
            "usage": {
                "prompt_tokens": len(prompt.split()),
                "completion_tokens": len(response.split()),
                "total_tokens": len(prompt.split()) + len(response.split())
            }
        })

    except Exception as e:
        print(f"❌ Error in /generate: {str(e)}")
        return jsonify({"error": f"Generation failed: {str(e)}"}), 500

print("✅ /generate endpoint added successfully!")

In [ ]:
# Cell 5: LLM loading and setup
# Load a truly public and open-source LLM for demonstration purposes
# Note: This model is not optimized for RAG or chat, but allows the code to run without authentication issues.
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# BitsAndBytesConfig is not applicable to gpt2 in the same way as Llama
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

In [ ]:
# Cell 6: RAG pipeline implementation

# Instantiate SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
document_embeddings = embedding_model.encode(knowledge_base)

# Create FAISS index
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# Define prompt template (using the one from PROMPTS dictionary)
prompt_template = PROMPTS["base_retrieval_prompt"]

# Implement retrieval and generation function
def generate_response(query):
    # Retrieve relevant documents
    query_embedding = embedding_model.encode([query])
    distance, index_id = index.search(query_embedding, k=2) # Retrieve top 2 documents
    retrieved_documents = [knowledge_base[i] for i in index_id[0]]

    # Format the prompt
    context = "\n".join(retrieved_documents)
    prompt = prompt_template.format(context=context, question=query)

    # Generate response using the LLM
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Set pad_token_id for GPT2 if not already set
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    outputs = model.generate(**input_ids, max_new_tokens=150, pad_token_id=tokenizer.pad_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up the response to remove the prompt part.
    # GPT2 might not follow the prompt structure perfectly, so splitting by "Answer:" might not always work.
    # A more robust approach for GPT2 might require more complex parsing or fine-tuning.
    try:
      response = response.split("Answer:")[-1].strip()
    except IndexError:
      # If "Answer:" is not found, return the full generated text after the prompt
      prompt_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
      if response.startswith(prompt_text):
          response = response[len(prompt_text):].strip()
      else:
          response = response.strip() # Fallback

    return response

In [ ]:
# Cell 7: Flask API setup

@app.route('/ping', methods=['GET'])
def ping():
    # Note: Ping endpoint for server status, not direct LLM interaction as per original request
    return jsonify({"status": "server is alive"})

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "server is healthy"})

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        if not data or 'query' not in data:
            return jsonify({"error": "Invalid request: 'query' field is required"}), 400

        user_query = data['query']
        response_text = generate_response(user_query)

        return jsonify({"response": response_text})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
# Cell 8: ngrok token input and tunnel creation

# Get ngrok authtoken from user input
authtoken = input("Enter your ngrok authtoken from https://dashboard.ngrok.com/get-started/your-authtoken: ")

# Set the ngrok authtoken
ngrok.set_auth_token(authtoken)

# Run the Flask app with ngrok
# This will block the cell execution until the server is stopped.
run_with_ngrok(app)

In [ ]:
# Cell 9: Testing and validation (Optional)

# You can add code here to test your Flask API endpoints
# For example, using the 'requests' library

# import requests
# import json

# Replace with the ngrok URL provided after running the previous cell
# ngrok_url = "YOUR_NGROK_URL"

# Test /ping endpoint
# try:
#     ping_response = requests.get(f"{ngrok_url}/ping")
#     print(f"/ping status: {ping_response.status_code}")
#     print(f"/ping response: {ping_response.json()}")
# except requests.exceptions.RequestException as e:
#     print(f"Error testing /ping: {e}")

# Test /health endpoint
# try:
#     health_response = requests.get(f"{ngrok_url}/health")
#     print(f"/health status: {health_response.status_code}")
#     print(f"/health response: {health_response.json()}")
# except requests.exceptions.RequestException as e:
#     print(f"Error testing /health: {e}")

# Test /chat endpoint
# try:
#     chat_query = {"query": "What is the capital of France?"}
#     chat_response = requests.post(f"{ngrok_url}/chat", json=chat_query)
#     print(f"/chat status: {chat_response.status_code}")
#     print(f"/chat response: {chat_response.json()}")
# except requests.exceptions.RequestException as e:
#     print(f"Error testing /chat: {e}")